**Steps and interpretation**

1- Data Preprocessing 

2- Tokenization using Spacy and Bert model 

3- take top 1000 token from each tokenizer accordance to thier entropy value ( coded it from scratch)

4- Found overlap percentage of taken among two tokenizers

4- plotted freq of token vs entropy value of unique token in both sets/lists which give interpretation that *SpaCy have a slightly higher peak in the very low entropy range ( more no. of unique token having low entropy) and very less no of token at other higher entropy level as compared to BERT tokenizer.In Bert Tokenizer we see that there is good spread of token at consecutive higher level of entropy. 


**1- Download dependencies**

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from bs4 import BeautifulSoup
from datasets import load_dataset
import re
import string
from textblob import TextBlob
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
#Setting English stopwords
stopword = nltk.corpus.stopwords.words('english')

e:\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amrit/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**2- Preprocessing text**

In [3]:
# Convert text to lower case
def convert_lower(text):
    return text.lower()

# Remove <html> tag
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


# #Removing the square brackets
def remove_bw_square_bracket(text):
    return re.sub('\[[^]]*\]', '', text)


# Remove URL 
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'', text)


# Remove punctuation  ( '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~')
punc = string.punctuation
# punc = string.punctuation.replace("'", "")

def remove_punc(text):
    return text.translate(str.maketrans('', '', punc))


# correct text using TextBlob   ( ceertain ---> certain)
def correct_text(text):
    text_blob = TextBlob(text)
    return text_blob.correct().string


# Remove stopword in the text
def remove_stopwords(text):
    new_text = []
    
    for word in text.split():
        if word in stopword:
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)


chat_words = {
    "AFAIK": "As Far As I Know",
    "AFK": "Away From Keyboard",
    "ASAP": "As Soon As Possible",
    "ATK": "At The Keyboard",
    "ATM": "At The Moment",
    "A3": "Anytime, Anywhere, Anyplace",
    "BAK": "Back At Keyboard",
    "BBL": "Be Back Later",
    "BBS": "Be Back Soon",
    "BFN": "Bye For Now",
    "B4N": "Bye For Now",
    "BRB": "Be Right Back",
    "BRT": "Be Right There",
    "BTW": "By The Way",
    "B4": "Before",
    "B4N": "Bye For Now",
    "CU": "See You",
    "CUL8R": "See You Later",
    "CYA": "See You",
    "FAQ": "Frequently Asked Questions",
    "FC": "Fingers Crossed",
    "FWIW": "For What It's Worth",
    "FYI": "For Your Information",
    "GAL": "Get A Life",
    "GG": "Good Game",
    "GN": "Good Night",
    "GMTA": "Great Minds Think Alike",
    "GR8": "Great!",
    "G9": "Genius",
    "IC": "I See",
    "ICQ": "I Seek you (also a chat program)",
    "ILU": "ILU: I Love You",
    "IMHO": "In My Honest/Humble Opinion",
    "IMO": "In My Opinion",
    "IOW": "In Other Words",
    "IRL": "In Real Life",
    "KISS": "Keep It Simple, Stupid",
    "LDR": "Long Distance Relationship",
    "LMAO": "Laugh My A.. Off",
    "LOL": "Laughing Out Loud",
    "LTNS": "Long Time No See",
    "L8R": "Later",
    "MTE": "My Thoughts Exactly",
    "M8": "Mate",
    "NRN": "No Reply Necessary",
    "OIC": "Oh I See",
    "PITA": "Pain In The A..",
    "PRT": "Party",
    "PRW": "Parents Are Watching",
    "QPSA?": "Que Pasa?",
    "ROFL": "Rolling On The Floor Laughing",
    "ROFLOL": "Rolling On The Floor Laughing Out Loud",
    "ROTFLMAO": "Rolling On The Floor Laughing My A.. Off",
    "SK8": "Skate",
    "STATS": "Your sex and age",
    "ASL": "Age, Sex, Location",
    "THX": "Thank You",
    "TTFN": "Ta-Ta For Now!",
    "TTYL": "Talk To You Later",
    "U": "You",
    "U2": "You Too",
    "U4E": "Yours For Ever",
    "WB": "Welcome Back",
    "WTF": "What The F...",
    "WTG": "Way To Go!",
    "WUF": "Where Are You From?",
    "W8": "Wait...",
    "7K": "Sick:-D Laugher",
    "TFW": "That feeling when",
    "MFW": "My face when",
    "MRW": "My reaction when",
    "IFYP": "I feel your pain",
    "TNTL": "Trying not to laugh",
    "JK": "Just kidding",
    "IDC": "I don't care",
    "ILY": "I love you",
    "IMU": "I miss you",
    "ADIH": "Another day in hell",
    "ZZZ": "Sleeping, bored, tired",
    "WYWH": "Wish you were here",
    "TIME": "Tears in my eyes",
    "BAE": "Before anyone else",
    "FIMH": "Forever in my heart",
    "BSAAW": "Big smile and a wink",
    "BWL": "Bursting with laughter",
    "BFF": "Best friends forever",
    "CSL": "Can't stop laughing"
}

# Acroynmys handling 
def chat_conversion(text):
    new_text = []
    for i in text.split():
        if i.upper() in chat_words:
            new_text.append(chat_words[i.upper()])
        else:
            new_text.append(i)
    return " ".join(new_text)

contractions = {
    "doesn't": "does not",
    "don't": "do not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "can't": "cannot",
    "couldn't": "could not",
    "shouldn't": "should not",
    "wouldn't": "would not",
    "won't": "will not",
    "I'm": "I am",
    "I've": "I have",
    "I'd": "I would",
    "I'll": "I will",
    "you're": "you are",
    "you've": "you have",
    "you'd": "you would",
    "you'll": "you will",
    "he's": "he is",
    "he'd": "he would",
    "he'll": "he will",
    "she's": "she is",
    "she'd": "she would",
    "she'll": "she will",
    "it's": "it is",
    "it'd": "it would",
    "it'll": "it will",
    "we're": "we are",
    "we've": "we have",
    "we'd": "we would",
    "we'll": "we will",
    "they're": "they are",
    "they've": "they have",
    "they'd": "they would",
    "they'll": "they will",
    "that's": "that is",
    "there's": "there is",
    "there'd": "there would",
    "there'll": "there will",
    "who's": "who is",
    "who'd": "who would",
    "who'll": "who will",
    "what's": "what is",
    "what're": "what are",
    "what've": "what have",
    "what'd": "what would",
    "what'll": "what will",
    "let's": "let us",
    "how's": "how is",
    "how'd": "how would",
    "how'll": "how will",
    "here's": "here is",
    "she's": "she is",
    "he's": "he is",
    "we're": "we are"
}

# Function to replace contractions
def replace_contractions(text):
    words = text.split()
    replaced_text = []
    for word in words:
        if word in contractions:
            replaced_text.append(contractions[word])
        else:
            replaced_text.append(word)
    return " ".join(replaced_text)



<>:13: SyntaxWarning: invalid escape sequence '\['
<>:13: SyntaxWarning: invalid escape sequence '\['
C:\Users\amrit\AppData\Local\Temp\ipykernel_67252\3692572510.py:13: SyntaxWarning: invalid escape sequence '\['
  return re.sub('\[[^]]*\]', '', text)


In [4]:
def clean_text(text):
    text = convert_lower(text)
    text = strip_html(text)
    text = remove_bw_square_bracket(text)
    text = remove_url(text)
    text = remove_punc(text)
    # text = replace_contractions(text)
    text = remove_stopwords(text)
    text = chat_conversion(text)
    # text = correct_text(text)
    return text 

**Load Give dataset from huggingface  save it as df(csv)**

In [6]:
imdb_dataset = load_dataset("imdb")

# Convert the dataset to a pandas DataFrame
train_df = pd.DataFrame(imdb_dataset['train'])
test_df = pd.DataFrame(imdb_dataset['test'])

# Save the DataFrame to CSV files
train_df.to_csv('imdb1_train.csv', index=False)
# test_df.to_csv('imdb1_test.csv', index=False)

print("Dataset saved to CSV files.")

Dataset saved to CSV files.


In [7]:
# we will be using only train dataset for experiment

data= pd.read_csv('imdb1_train.csv')
data.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [8]:
data['text'] = data['text'].apply(clean_text)

C:\Users\amrit\AppData\Local\Temp\ipykernel_67252\3692572510.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [9]:
data.head()

,text,label
0,rented curiousyellow video store controversy s...,0
1,curious yellow risible pretentious steaming pi...,0
2,avoid making type film future film interesting...,0
3,film probably inspired godards masculin fémini...,0
4,oh brotherafter hearing ridiculous film umptee...,0


In [10]:
cleaned_text= data['text'].apply(clean_text).tolist()
cleaned_text

C:\Users\amrit\AppData\Local\Temp\ipykernel_67252\3692572510.py:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


['rented curiousyellow video store controversy surrounded first released 1967 also heard first seized us customs ever tried enter country therefore fan films considered controversial really see myselfthe plot centered around young swedish drama student named lena wants learn everything life particular wants focus attentions making sort documentary average swede thought certain political issues vietnam war race issues united states asking politicians ordinary denizens stockholm opinions politics sex drama teacher classmates married menwhat kills curiousyellow 40 years ago considered pornographic really sex nudity scenes far even shot like cheaply made porno countrymen mind find shocking reality sex nudity major staple swedish cinema even ingmar bergman arguably answer good old boy john ford sex scenes filmsi commend filmmakers fact sex shown film shown artistic purposes rather shock people make money shown pornographic theaters america curiousyellow good film anyone wanting study meat p

In [11]:
labels = data['label'].tolist()
# labels
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

**Load spacy and Bert tokenizer**

In [12]:
# Load tokenizers
from transformers import BertTokenizer
nlp = spacy.load("en_core_web_sm", disable=["ner", "tagger", "parser"])
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


**Tokenize the dataset**

In [13]:
# Tokenize the dataset
def tokenize_with_spacy(texts):
    return [token.text for doc in nlp.pipe(texts, batch_size=50) for token in doc]

def tokenize_with_bert(texts):
    return [token for text in texts for token in bert_tokenizer.tokenize(text)]

In [14]:
spacy_tokens = tokenize_with_spacy(cleaned_text)
spacy_tokens

e:\Python\Lib\site-packages\spacy\pipeline\lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


['rented',
 'curiousyellow',
 'video',
 'store',
 'controversy',
 'surrounded',
 'first',
 'released',
 '1967',
 'also',
 'heard',
 'first',
 'seized',
 'us',
 'customs',
 'ever',
 'tried',
 'enter',
 'country',
 'therefore',
 'fan',
 'films',
 'considered',
 'controversial',
 'really',
 'see',
 'myselfthe',
 'plot',
 'centered',
 'around',
 'young',
 'swedish',
 'drama',
 'student',
 'named',
 'lena',
 'wants',
 'learn',
 'everything',
 'life',
 'particular',
 'wants',
 'focus',
 'attentions',
 'making',
 'sort',
 'documentary',
 'average',
 'swede',
 'thought',
 'certain',
 'political',
 'issues',
 'vietnam',
 'war',
 'race',
 'issues',
 'united',
 'states',
 'asking',
 'politicians',
 'ordinary',
 'denizens',
 'stockholm',
 'opinions',
 'politics',
 'sex',
 'drama',
 'teacher',
 'classmates',
 'married',
 'menwhat',
 'kills',
 'curiousyellow',
 '40',
 'years',
 'ago',
 'considered',
 'pornographic',
 'really',
 'sex',
 'nudity',
 'scenes',
 'far',
 'even',
 'shot',
 'like',
 'cheapl

In [15]:
bert_tokens = tokenize_with_bert(cleaned_text)
bert_tokens

['rented',
 'curious',
 '##ye',
 '##llo',
 '##w',
 'video',
 'store',
 'controversy',
 'surrounded',
 'first',
 'released',
 '1967',
 'also',
 'heard',
 'first',
 'seized',
 'us',
 'customs',
 'ever',
 'tried',
 'enter',
 'country',
 'therefore',
 'fan',
 'films',
 'considered',
 'controversial',
 'really',
 'see',
 'myself',
 '##the',
 'plot',
 'centered',
 'around',
 'young',
 'swedish',
 'drama',
 'student',
 'named',
 'lena',
 'wants',
 'learn',
 'everything',
 'life',
 'particular',
 'wants',
 'focus',
 'attention',
 '##s',
 'making',
 'sort',
 'documentary',
 'average',
 'sw',
 '##ede',
 'thought',
 'certain',
 'political',
 'issues',
 'vietnam',
 'war',
 'race',
 'issues',
 'united',
 'states',
 'asking',
 'politicians',
 'ordinary',
 'den',
 '##ize',
 '##ns',
 'stockholm',
 'opinions',
 'politics',
 'sex',
 'drama',
 'teacher',
 'classmates',
 'married',
 'men',
 '##w',
 '##hat',
 'kills',
 'curious',
 '##ye',
 '##llo',
 '##w',
 '40',
 'years',
 'ago',
 'considered',
 'pornogra

In [16]:
import math

# Function to calculate entropy
def calculate_entropy(tokens):
    frequency = Counter(tokens)
    total_tokens = len(tokens)
    probabilities = [freq / total_tokens for freq in frequency.values()]
    entropy = -sum(p * math.log2(p) for p in probabilities)
    return entropy

In [17]:
# Calculate entropy for each tokenized text
spacy_entropies = [calculate_entropy(tokens) for tokens in spacy_tokens]
spacy_entropies

[2.2516291673878226,
 3.238901256602631,
 2.321928094887362,
 2.321928094887362,
 3.095795255000934,
 2.721928094887362,
 2.321928094887362,
 2.4056390622295662,
 2.0,
 2.0,
 2.321928094887362,
 2.321928094887362,
 2.2516291673878226,
 1.0,
 2.521640636343318,
 1.5,
 2.321928094887362,
 1.9219280948873623,
 2.8073549220576037,
 2.4193819456463714,
 1.584962500721156,
 2.321928094887362,
 2.9219280948873623,
 3.3927474104487847,
 2.2516291673878226,
 0.9182958340544896,
 2.94770277922009,
 2.0,
 2.4056390622295662,
 2.584962500721156,
 2.321928094887362,
 2.521640636343318,
 1.9219280948873623,
 2.521640636343318,
 2.321928094887362,
 2.0,
 2.321928094887362,
 2.321928094887362,
 3.1219280948873624,
 2.0,
 2.9219280948873623,
 2.321928094887362,
 2.321928094887362,
 2.6464393446710157,
 2.584962500721156,
 2.0,
 3.4594316186372973,
 2.2359263506290326,
 1.9219280948873623,
 2.2359263506290326,
 2.8073549220576037,
 2.725480556997868,
 1.792481250360578,
 2.8073549220576037,
 1.584962500

In [18]:
# Calculate entropy for each tokenized text
bert_entropies = [calculate_entropy(tokens) for tokens in bert_tokens]
bert_entropies

[2.2516291673878226,
 2.521640636343318,
 1.5,
 1.5219280948873621,
 0.9182958340544896,
 2.321928094887362,
 2.321928094887362,
 3.095795255000934,
 2.721928094887362,
 2.321928094887362,
 2.4056390622295662,
 2.0,
 2.0,
 2.321928094887362,
 2.321928094887362,
 2.2516291673878226,
 1.0,
 2.521640636343318,
 1.5,
 2.321928094887362,
 1.9219280948873623,
 2.8073549220576037,
 2.4193819456463714,
 1.584962500721156,
 2.321928094887362,
 2.9219280948873623,
 3.3927474104487847,
 2.2516291673878226,
 0.9182958340544896,
 2.584962500721156,
 1.9219280948873623,
 2.0,
 2.4056390622295662,
 2.584962500721156,
 2.321928094887362,
 2.521640636343318,
 1.9219280948873623,
 2.521640636343318,
 2.321928094887362,
 2.0,
 2.321928094887362,
 2.321928094887362,
 3.1219280948873624,
 2.0,
 2.9219280948873623,
 2.321928094887362,
 2.321928094887362,
 2.4193819456463714,
 0.9182958340544896,
 2.584962500721156,
 2.0,
 3.4594316186372973,
 2.2359263506290326,
 1.0,
 1.5219280948873621,
 2.235926350629032

In [19]:
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [20]:
model_name="distilbert/distilbert-base-uncased-finetuned-sst-2-english"
model= AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)

# Sentiment analysis using a pre-trained model
sentiment_analyzer = pipeline("sentiment-analysis",model=model,tokenizer=tokenizer)

# Analyze sentiment and print results along with entropy
# for text, entropy in zip(spacy_tokens[:1000], spacy_entropies[:1000]):
#     sentiment = sentiment_analyzer(text)[0]
#     print(f"Text: {text}")
#     print(f"Entropy: {entropy}")
#     print(f"Sentiment: {sentiment['label']}, Score: {sentiment['score']}")
#     print()

In [22]:
# Function to calculate weighted average sentiment score
def calculate_weighted_average_sentiment(sentiment_results, entropies):
    total_score = sum(result['score'] * entropy for result, entropy in zip(sentiment_results, entropies))
    total_entropy = sum(entropies)
    return total_score / total_entropy if total_entropy != 0 else 0


# Check if the entropy data is a list of lists or just a list of floats
if all(isinstance(e, list) for e in spacy_entropies):
    # Case: spacy_entropies is a list of lists (for multiple texts)
    top_1000_tokens_with_entropies = []

    for tokens, entropies in zip(spacy_tokens, spacy_entropies):
        # Pair each token with its corresponding entropy
        token_entropy_pairs = list(zip(tokens, entropies))

        # Sort tokens by entropy in descending order and select the top 1000
        sorted_token_entropy_pairs = sorted(token_entropy_pairs, key=lambda x: x[1], reverse=True)[:1000]

        # Add the top 1000 pairs to the list
        top_1000_tokens_with_entropies.extend(sorted_token_entropy_pairs)
else:
    # Case: spacy_entropies is a list of floats (for a single text)
    token_entropy_pairs = list(zip(spacy_tokens, spacy_entropies))

    # Sort tokens by entropy in descending order and select the top 1000
    top_1000_tokens_with_entropies = sorted(token_entropy_pairs, key=lambda x: x[1], reverse=True)[:1000]

# Analyze sentiment for the top 1000 tokens and calculate the weighted average sentiment score
top_1000_tokens = [token for token, entropy in top_1000_tokens_with_entropies]
top_1000_entropies = [entropy for token, entropy in top_1000_tokens_with_entropies]

sentiment_results = [sentiment_analyzer(token)[0] for token in top_1000_tokens]

# Calculate the weighted average sentiment score
weighted_average_sentiment_score = calculate_weighted_average_sentiment(sentiment_results, top_1000_entropies)

# Print the weighted average sentiment score
print(f"Weighted Average Sentiment Score for spacy top 1000 tokens: {weighted_average_sentiment_score}")


Weighted Average Sentiment Score for spacy top 1000 tokens: 0.9605632289684914


In [25]:
# Function to calculate weighted average sentiment score
def calculate_weighted_average_sentiment(sentiment_results, entropies):
    total_score = sum(result['score'] * entropy for result, entropy in zip(sentiment_results, entropies))
    total_entropy = sum(entropies)
    return total_score / total_entropy if total_entropy != 0 else 0


# Check if the entropy data is a list of lists or just a list of floats
if all(isinstance(e, list) for e in bert_entropies):
    # Case: bert_entropies is a list of lists (for multiple texts)
    top_1000_tokens_with_entropies = []

    for tokens, entropies in zip(bert_tokens, bert_entropies):
        # Pair each token with its corresponding entropy
        token_entropy_pairs = list(zip(tokens, entropies))

        # Sort tokens by entropy in descending order and select the top 1000
        sorted_token_entropy_pairs = sorted(token_entropy_pairs, key=lambda x: x[1], reverse=True)[:1000]

        # Add the top 1000 pairs to the list
        top_1000_tokens_with_entropies.extend(sorted_token_entropy_pairs)
else:
    # Case: bert is a list of floats (for a single text)
    token_entropy_pairs = list(zip(bert_tokens, bert_entropies))

    # Sort tokens by entropy in descending order and select the top 1000
    top_1000_tokens_with_entropies = sorted(token_entropy_pairs, key=lambda x: x[1], reverse=True)[:1000]

# Analyze sentiment for the top 1000 tokens and calculate the weighted average sentiment score
top_1000_tokens = [token for token, entropy in top_1000_tokens_with_entropies]
top_1000_entropies = [entropy for token, entropy in top_1000_tokens_with_entropies]

sentiment_results = [sentiment_analyzer(token)[0] for token in top_1000_tokens]

# Calculate the weighted average sentiment score
weighted_average_sentiment_score = calculate_weighted_average_sentiment(sentiment_results, top_1000_entropies)

# Print the weighted average sentiment score
print(f"Weighted Average Sentiment Score for bert top 1000 tokens: {weighted_average_sentiment_score}")


ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).